In [1]:
import warnings                        
warnings.filterwarnings('ignore')   
import requests
import folium
from pandas.io.json import json_normalize

In [2]:
# 전국 또는 특정 지역의 스타벅스 매장 위치를 찾아와서 지도위에 표시하기
# 1. requests로 스타벅스 위치 데이터를 가져와서 딕셔너리 타입으로 변환한다.
# 2. json_normalize() 메소드로 json 타입이 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# 3. folium 모듈을 사용해서 지도를 표시하고 지도위의 스타벅스 매장에 마커를 표시한다.

In [27]:
# 서울 스타벅스 전 지점

targetSize = 'https://www.starbucks.co.kr/store/getStore.do?r=4SN84R4DCR'
request = requests.post(targetSize,data={

    'ins_lat': 37.4931456, # 위도 
    'ins_lng': 127.0317056, # 경도
    'p_sido_cd': '01', # 시도 코드
    'p_gugun_cd': '0101', # 구군 코드
    'in_biz_cd':'', # ? 
    'iend': 1600, # 서버가 응답하는 최대 매장의 개수
    'set_date': ''# ?
    # 'in_biz_cd', 'set_date'를 넘겨주지 않으면 서버에서 데이터를 얻어오지 못한다.
})
storeList = request.json()
print(len(storeList['list']))


83


In [15]:
star_json = request.json() # 딕셔너리 타입으로 바뀐다.
print(type(star_json)) # <class 'dict'>
print(len(star_json)) # 1
#print(star_json)

<class 'dict'>
1


In [9]:
# pandas.io.json모듈의 import json_normalize() 메소드로 json 타입이 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# json_normalize(데이터프레임으로 변환할 데이터가 저장된 딕셔너리, '딕셔너리 중에서 ')
star_df = json_normalize(star_json, 'list')
print(type(star_df))
print(len(star_df))

<class 'pandas.core.frame.DataFrame'>
544


In [11]:
#star_df

In [14]:
# 작업에 필요한 컬럼 몇가지를 선택해서 지도에 마커를 표시할 때 사용할 데이터가 저장된 데이터프레임을 만든다.
# s_name -> 지점이름
# sido_code -> 시도 코드
# sido_name -> 시도 이름
# gugun_code -> 구군 코드
# gugun_name -> 구군 이름
# doro_address -> 도로명 주소
# lat -> 위도
# lot -> 경도

# 판다스 데이터프레임에서 특정 컬럼의 데이터를 얻어올 때 얻어올 컬럼이 1개라면 컬럼 이름만 적으면 되지만 얻어올 컬럼이 2개 
# 이상일 경우 얻어올 컬럼 이름을 리스트로 만들어 얻어와야 한다.
star_df_map = star_df[['s_name','sido_code','sido_code','gugun_code','gugun_name','doro_address','lat','lot']]
star_df_map

,s_name,sido_code,sido_code,gugun_code,gugun_name,doro_address,lat,lot
0,역삼아레나빌딩,01,01,0101,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,논현역사거리,01,01,0101,강남구,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,신사역성일빌딩,01,01,0101,강남구,서울특별시 강남구 강남대로 584 (논현동),37.514132,127.020563
3,국기원사거리,01,01,0101,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,스탈릿대치R,01,01,0101,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...,...,...,...,...
539,사가정역,01,01,0125,중랑구,서울특별시 중랑구 면목로 310,37.579594,127.087966
540,상봉역,01,01,0125,중랑구,"서울특별시 중랑구 망우로 307, ,3,4번지 (상봉동)",37.59689,127.08647
541,묵동이마트,01,01,0125,중랑구,"서울특별시 중랑구 동일로 932 (묵동, 묵동자이아파트) (묵동이마트 B1층)",37.613433,127.077484
542,묵동,01,01,0125,중랑구,서울특별시 중랑구 동일로 952,37.615368,127.076633


In [16]:
star_df_map.dtypes

s_name          object
sido_code       object
sido_code       object
gugun_code      object
gugun_name      object
doro_address    object
lat             object
lot             object
dtype: object

In [17]:
# astype() 메소드로 위도, 경도의 데이터 타입을 oject에서 float으로 변경한다.
star_df_map['lat'] = star_df_map['lat'].astype(float)
star_df_map['lot'] = star_df_map['lot'].astype(float)
star_df_map.dtypes

s_name           object
sido_code        object
sido_code        object
gugun_code       object
gugun_name       object
doro_address     object
lat             float64
lot             float64
dtype: object